In [53]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st

### Обрабатываем данные

In [2]:
f = open('Москва_2021.txt', "r")
data = []
for line in f:
    line = line.strip()
    if line:  
        number = int(line) 
        data.append(number)

In [3]:
data[:10]

[23, 32, 27, 26, 44, 49, 63, 34, 66, 66]

### Определяем объем выборки и выборочные среднее для каждой выборки

In [56]:
gamma = 0.95
delta = 3
t = st.t.ppf((1 + gamma) / 2, len(data))
sigma = np.std(data)

n = int(np.ceil(t**2 * sigma**2 / delta**2))

samples = []
means_of_samples = []
for i in range(36):
    sample = np.random.choice(data, size=n, replace=True)
    samples.append(sample)
    sample_mean = np.mean(sample)
    means_of_samples.append(sample_mean)

print("Объем каждой выборки:", n)
print("Выборочные средние для каждой выборки:", means_of_samples)

Объем каждой выборки: 62
Выборочные средние для каждой выборки: [34.37096774193548, 34.03225806451613, 36.96774193548387, 33.903225806451616, 36.354838709677416, 31.483870967741936, 37.79032258064516, 36.24193548387097, 31.903225806451612, 35.0, 37.24193548387097, 34.58064516129032, 34.87096774193548, 34.08064516129032, 34.25806451612903, 33.33870967741935, 36.12903225806452, 37.403225806451616, 32.096774193548384, 33.08064516129032, 36.41935483870968, 34.306451612903224, 32.354838709677416, 33.25806451612903, 36.693548387096776, 38.67741935483871, 32.483870967741936, 34.82258064516129, 33.37096774193548, 36.45161290322581, 33.96774193548387, 32.04838709677419, 34.645161290322584, 34.41935483870968, 35.645161290322584, 37.225806451612904]


## 1. По критерию Пирсона при уровне значимости α = 0,05 проверить нулевую гипотезу о нормальном распределении


In [57]:
alpha = 0.05

### a. Случайной величины «возраст». Для этого разбить массив исходных данных на 7 групп с равными интервалами (концы интервала – целые).

In [80]:
hist, bin_edges = np.histogram(data, bins=7)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
print('Интервалы: ', bin_centers)
print('Шаг: ', bin_centers[1] - bin_centers[0])
# Вычисляем ожидаемые частоты для нормального распределения
mean, std = np.mean(data), np.std(data)
expected = st.norm.pdf(bin_centers, mean, std) # st.norm.pdf - вычисляет Z значение
expected = expected / sum(expected) * len(data)
# Выполняем тест хи-квадрат
chi2, p_value = st.chisquare(hist, expected)

print(f"Тест нормальности для возраста (критерий Пирсона):")
print(f"Хи-квадрат статистика: {chi2}")
print(f"P-значение: {p_value}")
print(f"{'Отвергаем' if p_value < alpha else 'Не отвергаем'} нулевую гипотезу о нормальном распределении.")

Интервалы:  [18.21428571 26.64285714 35.07142857 43.5        51.92857143 60.35714286
 68.78571429]
Шаг:  8.42857142857143
Тест нормальности для возраста (критерий Пирсона):
Хи-квадрат статистика: 2903.5525084801925
P-значение: 0.0
Отвергаем нулевую гипотезу о нормальном распределении.


### б. Случайной величины «средний возраст», используя выборку средних, полученную в лабораторной работе №2.

In [81]:
min_mean = np.floor(min(means_of_samples))
max_mean = np.ceil(max(means_of_samples))
bins = np.arange(min_mean, max_mean + 1, 1)

hist, bin_edges = np.histogram(means_of_samples, bins=bins)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
print("Интервалы:", bin_centers)
print('Шаг: ', bin_centers[1] - bin_centers[0])
# Вычисляем параметры нормального распределения
mean = np.mean(means_of_samples)
std = np.std(means_of_samples, ddof=1)

# Вычисляем ожидаемые частоты
expected = st.norm.pdf(bin_centers, mean, std)
expected = expected / sum(expected) * len(means_of_samples)

# Выполняем тест хи-квадрат
chi2, p_value = st.chisquare(hist, expected)

print(f"Тест нормальности для среднего возраста (критерий Пирсона):")
print(f"Хи-квадрат статистика: {chi2}")
print(f"P-значение: {p_value}")
print(f"{'Отвергаем' if p_value < alpha else 'Не отвергаем'} нулевую гипотезу о нормальном распределении.")

Интервалы: [31.5 32.5 33.5 34.5 35.5 36.5 37.5 38.5]
Шаг:  1.0
Тест нормальности для среднего возраста (критерий Пирсона):
Хи-квадрат статистика: 5.798622351165335
P-значение: 0.563450299236695
Не отвергаем нулевую гипотезу о нормальном распределении.


## 2. По двум сгенерированным выборкам из лабораторной работы №2 проверить нулевую гипотезу о равенстве дисперсий генеральных совокупностей при уровне значимости 0,05 при конкурирующей гипотезе:


In [82]:
f_value = np.var(samples[0], ddof=1) / np.var(samples[1], ddof=1)
df1, df2 = len(samples[0]) - 1, len(samples[1]) - 1
p_value = 2 * min(st.f.cdf(f_value, df1, df2), 1 - st.f.cdf(f_value, df1, df2))

print(f"\nТест равенства дисперсий:")
print(f"F-статистика: {f_value}")
print(f"P-значение: {p_value}")
print(f"{'Отвергаем' if p_value < alpha else 'Не отвергаем'} нулевую гипотезу о равенстве дисперсий.")


Тест равенства дисперсий:
F-статистика: 1.5284504957432645
P-значение: 0.10019875648140153
Не отвергаем нулевую гипотезу о равенстве дисперсий.


### a. Проверка H1: D1 > D2

In [66]:
p_value_greater = 1 - st.f.cdf(f_value, df1, df2)
print(f"\nПроверка H1: D1 > D2:")
print(f"P-значение: {p_value_greater}")
print(f"{'Отвергаем нулевую гипотезу в пользу H1: D1 > D2.' if p_value < alpha else 'Не отвергаем нулевую гипотезу'}")


Проверка H1: D1 > D2:
P-значение: 0.050099378240700765
Не отвергаем нулевую гипотезу


### b. Проверка H1: D1 ≠ D2

In [67]:
print(f"\nПроверка H1: D1 ≠ D2:")
print(f"P-значение: {p_value}")
print(f"{'Отвергаем нулевую гипотезу в пользу H1: D1 ≠ D2.' if p_value < alpha else 'Не отвергаем нулевую гипотезу'}")


Проверка H1: D1 ≠ D2:
P-значение: 0.10019875648140153
Не отвергаем нулевую гипотезу
